# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Load NHL team performance data from CSV
nhl_merged_Nhl = pd.read_csv("nhl.csv")

# Load city population and sports data table from the Wikipedia HTML
cities = pd.read_html("wikipedia_data.html")[1]

# Remove the last row (which may be a summary or NaN row) and select relevant columns
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# Extract only the city name, population, and NHL team columns
cities_Nhl = cities.iloc[:, list(range(2)) + [-1]]

# Clean the NHL column: remove footnotes like [note 1] and replace em dashes with NaN
cities_Nhl.loc[:, 'NHL'] = cities_Nhl['NHL'].replace({r'\[note \d+\]': '', '—': np.nan}, regex=True)

# Rename columns for clarity
cities_Nhl = cities_Nhl.rename(columns={"NHL": 'team', "Population (2016 est.)[8]": "Population"})

# Filter only 2018 data from the NHL dataset
nhl_2018 = nhl_merged_Nhl.loc[nhl_merged_Nhl['year'] == 2018, ["team", 'W', "L"]]

# Remove rows that contain division headers or non-team rows
nhl_2018 = nhl_2018[~nhl_2018.astype(str).apply(lambda x: x.str.contains('Division', case=False)).any(axis=1)]

# Remove asterisks (used for playoff teams etc.) from team names
nhl_2018['team'] = nhl_2018['team'].replace(r'\*', value='', regex=True)

# Repeat filter to ensure clean data
nhl_2018 = nhl_2018[~nhl_2018.astype(str).apply(lambda x: x.str.contains('Division', case=False)).any(axis=1)]

# List of city names used to clean team names
cities = [
    "Tampa Bay", "Boston", "Toronto", "Florida", "Detroit", "Montreal", "Ottawa",
    "Buffalo", "Washington", "Pittsburgh", "Philadelphia", "Columbus", "New Jersey",
    "Carolina", "New York", "New York", "Nashville", "Winnipeg", "Minnesota",
    "Colorado", "St. Louis", "Dallas", "Chicago", "Vegas", "Los Angeles",
    "Calgary", "Edmonton", "Vancouver", "Arizona", "San Jose"
]

# Clean the team names by removing the city prefix
cleaned = []
for name in nhl_2018['team']:
    matched = False
    for city in cities:
        if name.startswith(city):
            cleaned.append(name.replace(city, '').strip())
            matched = True
            break
    if not matched:
        cleaned.append(name)

nhl_2018['team'] = cleaned

# Normalize team names for multi-team metropolitan areas
nhl_2018.replace({
    'Kings': 'Kings Ducks',
    'Anaheim Ducks': 'Kings Ducks',
    "Devils": "Rangers Islanders Devils",
    'Islanders': "Rangers Islanders Devils",
    "Rangers": "Rangers Islanders Devils"
}, inplace=True)

# Merge city and NHL team data on team name
merged_Nhl = pd.merge(cities_Nhl, nhl_2018, how='right', on='team')

# Calculate win-loss percentage
merged_Nhl["W-L%"] = merged_Nhl["W"].astype("int") / (merged_Nhl["W"].astype("int") + merged_Nhl["L"].astype("int"))

# Convert Population and W-L% to numeric
merged_Nhl["Population"] = merged_Nhl["Population"].astype("float")
merged_Nhl["W-L%"] = merged_Nhl["W-L%"].astype("float")

# Drop rows with any missing data
merged_Nhl = merged_Nhl.dropna()

# Group by metro area: get first population and mean W-L%
population_by_region_df = merged_Nhl.groupby('Metropolitan area')['Population'].first()
win_loss_by_region_df = merged_Nhl.groupby('Metropolitan area')['W-L%'].mean()

# Final function to compute correlation
def nhl_correlation(): 
    population_by_region = population_by_region_df
    win_loss_by_region = win_loss_by_region_df

    # Checks
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    # Return Pearson correlation
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [9]:
nhl_correlation()

0.012486162921209881

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Load NBA performance data
nba_merged_Nhl = pd.read_csv("nba.csv")

# Load Wikipedia city table (sports by metro area)
cities = pd.read_html("wikipedia_data.html")[1]

# Drop last row and select relevant columns
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# Extract city name, population, and NBA team columns
cities_NBA = cities.iloc[:, list(range(2)) + [-2]]  # NBA column is second-to-last

# Clean NBA team names: remove footnotes and em dashes
cities_NBA.loc[:, 'NBA'] = cities_NBA['NBA'].replace({r'\[note \d+\]': '', '—': np.nan}, regex=True)

# Rename columns for clarity
cities_NBA = cities_NBA.rename(columns={"NBA": 'team', "Population (2016 est.)[8]": "Population"})

# Simplify team names to their last word (e.g., "Los Angeles Lakers" → "Lakers")
cities_NBA['team'] = cities_NBA['team'].str.split().str[-1]

# Filter 2018 data only
nba_2018 = nba_merged_Nhl.loc[nba_merged_Nhl['year'] == 2018, ["team", 'W', "W/L%"]]

# Drop division headers or non-team rows
nba_2018 = nba_2018[~nba_2018.astype(str).apply(lambda x: x.str.contains('Division', case=False)).any(axis=1)]

# Clean playoff asterisks and seeding info (e.g., * or (1))
nba_2018['team'] = nba_2018['team'].replace({r'\*': '', r'\(\d*\)': ""}, regex=True)

# Repeat filter to ensure clean data
nba_2018 = nba_2018[~nba_2018.astype(str).apply(lambda x: x.str.contains('Division', case=False)).any(axis=1)]

# Simplify team names again
nba_2018['team'] = nba_2018['team'].str.split().str[-1]

# Normalize metro naming for shared metros
nba_2018.replace({
    'Lakers': 'Clippers',
    'Knicks': 'Nets',
}, inplace=True)

# Merge cleaned team data with cities on simplified team names
merged_nba = pd.merge(cities_NBA, nba_2018, how='right', on='team')

# Convert Population and Win/Loss to float
merged_nba["Population"] = merged_nba["Population"].astype("float")
merged_nba["W/L%"] = merged_nba["W/L%"].astype("float")

# Define correlation function
def nba_correlation():
    population_by_region = merged_nba.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = merged_nba.groupby('Metropolitan area')['W/L%'].mean()
    
    # Assertions to confirm correct setup
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    # Return Pearson correlation
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [15]:
nba_correlation()

-0.1763635064218294

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Load MLB performance data
mlb_merged_Nhl = pd.read_csv("mlb.csv")

# Load Wikipedia table (sports teams by metro area)
cities = pd.read_html("wikipedia_data.html")[1]

# Drop the last row and select relevant columns
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# Extract metro area name, population, and MLB team column
cities_MLB = cities.iloc[:, list(range(2)) + [-3]]  # MLB is third-to-last column

# Clean up MLB team names: remove footnotes and replace em dashes with NaN
cities_MLB.loc[:, 'MLB'] = cities_MLB['MLB'].replace({r'\[note \d+\]': '', '—': np.nan}, regex=True)

# Rename columns for clarity
cities_MLB = cities_MLB.rename(columns={"MLB": 'team', "Population (2016 est.)[8]": "Population"})

# Filter only 2018 season data
mlb_merged_Nhl = mlb_merged_Nhl.loc[mlb_merged_Nhl['year'] == 2018, ["team", "W-L%"]]

# Normalize team names to common metro representations
team = {
    'Boston Red Sox': 'Red Sox',
    'New York Yankees': 'Yankees Mets',
    'Toronto Blue Jays': 'Blue Jays',
    'Tampa Bay Rays': 'Rays',
    'Baltimore Orioles': 'Orioles',
    'Chicago White Sox': 'Cubs White Sox',
    'Cleveland Indians': 'Indians',
    'Detroit Tigers': 'Tigers',
    'Kansas City Royals': 'Royals',
    'Minnesota Twins': 'Twins',
    'Houston Astros': 'Astros',
    'Los Angeles Angels': 'Dodgers Angels',
    'Oakland Athletics': 'Giants Athletics',
    'Seattle Mariners': 'Mariners',
    'Texas Rangers': 'Rangers',
    'Atlanta Braves': 'Braves',
    'Miami Marlins': 'Marlins',
    'New York Mets': 'Yankees Mets',
    'Philadelphia Phillies': 'Phillies',
    'Washington Nationals': 'Nationals',
    'Chicago Cubs': 'Cubs White Sox',
    'Cincinnati Reds': 'Reds',
    'Milwaukee Brewers': 'Brewers',
    'Pittsburgh Pirates': 'Pirates',
    'St. Louis Cardinals': 'Cardinals',
    'Arizona Diamondbacks': 'Diamondbacks',
    'Colorado Rockies': 'Rockies',
    'Los Angeles Dodgers': 'Dodgers Angels',
    'San Francisco Giants': 'Giants Athletics',
    'San Diego Padres': 'Padres'
}

# Replace team names with standardized metro identifiers
mlb_merged_Nhl['team'] = mlb_merged_Nhl['team'].replace(team)

# Merge city and team data
merged_Mlb = pd.merge(cities_MLB, mlb_merged_Nhl, how='right', on='team')

# Convert columns to numeric
merged_Mlb["Population"] = merged_Mlb["Population"].astype("float")
merged_Mlb["W-L%"] = merged_Mlb["W-L%"].astype("float")

# Final function to calculate correlation
def mlb_correlation(): 
    population_by_region = merged_Mlb.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = merged_Mlb.groupby('Metropolitan area')['W-L%'].mean()
    
    # Assertions for testing
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    # Return Pearson correlation coefficient
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [23]:
mlb_correlation()

0.150037374754095

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [39]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Load NFL performance data
nfl_merged_Nhl = pd.read_csv("nfl.csv")

# Load city data
cities = pd.read_html("wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]  # Remove last summary row and keep relevant columns

# Extract metro name, population, and NFL team
cities_NFL = cities.iloc[:, list(range(3))]  # Includes metro, population, NFL

# Clean up the NFL column: remove footnotes and replace em dashes with NaN
cities_NFL.loc[:, 'NFL'] = cities_NFL['NFL'].replace({r'\[note \d+\]': '', '—': np.nan}, regex=True)

# Rename columns for clarity
cities_NFL = cities_NFL.rename(columns={"NFL": 'team', "Population (2016 est.)[8]": "Population"})


In [57]:
team = {
    'New England Patriots': 'Patriots',
    'Miami Dolphins': 'Dolphins',
    'Buffalo Bills': 'Bills',
    'New York Jets': 'Giants Jets',
    'Baltimore Ravens': 'Ravens',
    'Pittsburgh Steelers': 'Steelers',
    'Cleveland Browns': 'Browns',
    'Cincinnati Bengals': 'Bengals',
    'Houston Texans': 'Texans',
    'Indianapolis Colts': 'Colts',
    'Tennessee Titans': 'Titans',
    'Jacksonville Jaguars': 'Jaguars',
    'Kansas City Chiefs': 'Chiefs',
    'Los Angeles Chargers': 'Rams Chargers',
    'Denver Broncos': 'Broncos',
    'Oakland Raiders' : '49ers Raiders',
    'Dallas Cowboys': 'Cowboys',
    'Philadelphia Eagles': 'Eagles',
    'Washington Redskins' : 'Redskins',
    'New York Giants' : 'Giants Jets',
    'Chicago Bears': 'Bears',
    'Minnesota Vikings': 'Vikings',
    'Green Bay Packers': 'Packers',
    'Detroit Lions': 'Lions',
    'New Orleans Saints': 'Saints',
    'Carolina Panthers': 'Panthers',
    'Atlanta Falcons': 'Falcons',
    'Tampa Bay Buccaneers': 'Buccaneers',
    'Los Angeles Rams': 'Rams Chargers',
    'Seattle Seahawks': 'Seahawks',
    'San Francisco 49ers': '49ers Raiders',
    'Arizona Cardinals': 'Cardinals'
}
nfl_merged_Nhl = nfl_merged_Nhl.loc[nfl_merged_Nhl['year'] == 2018, ["team","W-L%"]]

nfl_merged_Nhl = nfl_merged_Nhl.loc[:,['team', 'W-L%']]





In [59]:
# List of words to filter out
words_to_remove = ['AFC', 'NFC']

# Create a regex pattern from the list
pattern = '|'.join(words_to_remove)

# Remove rows that contain any of the words
nfl_merged_Nhl = nfl_merged_Nhl[~nfl_merged_Nhl['team'].str.contains(pattern, case=False, na=False)].reset_index(drop=True)

In [61]:
nfl_merged_Nhl.replace({r'\*': '', r'\+': '', r'\.': ''}, regex=True, inplace=True)


In [63]:

nfl_merged_Nhl['team'] = nfl_merged_Nhl['team'].replace(team)

In [65]:
nfl_merged_Nhl

,team,W-L%
0,Patriots,688
1,Dolphins,438
2,Bills,375
3,Giants Jets,250
4,Ravens,625
5,Steelers,594
6,Browns,469
7,Bengals,375
8,Texans,688
9,Colts,625


In [67]:
merged_NFL= pd.merge(cities_NFL, nfl_merged_Nhl, how='right', on='team')

In [69]:
merged_NFL["Population"],merged_NFL["W-L%"] = merged_NFL["Population"].astype("float"),merged_NFL["W-L%"].astype("float")

In [71]:
population_by_region= merged_NFL.groupby('Metropolitan area')['Population'].first()
win_loss_by_region= merged_NFL.groupby('Metropolitan area')['W-L%'].mean()
    

In [73]:
merged_NFL

,Metropolitan area,Population,team,W-L%
0,Boston,4794447.0,Patriots,688.0
1,Miami–Fort Lauderdale,6066387.0,Dolphins,438.0
2,Buffalo,1132804.0,Bills,375.0
3,New York City,20153634.0,Giants Jets,250.0
4,Baltimore,2798886.0,Ravens,625.0
5,Pittsburgh,2342299.0,Steelers,594.0
6,Cleveland,2055612.0,Browns,469.0
7,Cincinnati,2165139.0,Bengals,375.0
8,Houston,6772470.0,Texans,688.0
9,Indianapolis,2004230.0,Colts,625.0


In [83]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Load data
nfl_merged_Nhl = pd.read_csv("nfl.csv")
cities = pd.read_html("wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# Filter city data for NFL
cities_NFL = cities.iloc[:, list(range(3))]
cities_NFL = cities_NFL.rename(columns={"NFL": 'team', "Population (2016 est.)[8]": "Population"})
cities_NFL.loc[:, 'team'] = cities_NFL['team'].replace({r'\[note \d+\]': '', '—': np.nan}, regex=True)

# Filter NFL data for 2018 and clean team names
nfl_merged_Nhl = nfl_merged_Nhl.loc[nfl_merged_Nhl['year'] == 2018, ["team", "W-L%"]]

# Remove rows with division headers like 'AFC' and 'NFC'
words_to_remove = ['AFC', 'NFC']
pattern = '|'.join(words_to_remove)
nfl_merged_Nhl = nfl_merged_Nhl[~nfl_merged_Nhl['team'].str.contains(pattern, case=False, na=False)].reset_index(drop=True)

# Clean special characters from team names
nfl_merged_Nhl.replace({r'\*': '', r'\+': '', r'\.': ''}, regex=True, inplace=True)

# Normalize team names for multi-team cities
team = {
    'New England Patriots': 'Patriots',
    'Miami Dolphins': 'Dolphins',
    'Buffalo Bills': 'Bills',
    'New York Jets': 'Giants Jets',
    'Baltimore Ravens': 'Ravens',
    'Pittsburgh Steelers': 'Steelers',
    'Cleveland Browns': 'Browns',
    'Cincinnati Bengals': 'Bengals',
    'Houston Texans': 'Texans',
    'Indianapolis Colts': 'Colts',
    'Tennessee Titans': 'Titans',
    'Jacksonville Jaguars': 'Jaguars',
    'Kansas City Chiefs': 'Chiefs',
    'Los Angeles Chargers': 'Rams Chargers',
    'Denver Broncos': 'Broncos',
    'Oakland Raiders': '49ers Raiders',
    'Dallas Cowboys': 'Cowboys',
    'Philadelphia Eagles': 'Eagles',
    'Washington Redskins': 'Redskins',
    'New York Giants': 'Giants Jets',
    'Chicago Bears': 'Bears',
    'Minnesota Vikings': 'Vikings',
    'Green Bay Packers': 'Packers',
    'Detroit Lions': 'Lions',
    'New Orleans Saints': 'Saints',
    'Carolina Panthers': 'Panthers',
    'Atlanta Falcons': 'Falcons',
    'Tampa Bay Buccaneers': 'Buccaneers',
    'Los Angeles Rams': 'Rams Chargers',
    'Seattle Seahawks': 'Seahawks',
    'San Francisco 49ers': '49ers Raiders',
    'Arizona Cardinals': 'Cardinals'
}
nfl_merged_Nhl['team'] = nfl_merged_Nhl['team'].replace(team)

# Merge with city population data
merged_NFL = pd.merge(cities_NFL, nfl_merged_Nhl, how='right', on='team')
merged_NFL["Population"] = merged_NFL["Population"].astype("float")
merged_NFL["W-L%"] = merged_NFL["W-L%"].astype("float")

# Final correlation function
def nfl_correlation(): 
    population_by_region = merged_NFL.groupby('Metropolitan area')['Population'].first()
    win_loss_by_region = merged_NFL.groupby('Metropolitan area')['W-L%'].mean()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

# Call function if needed
# print(nfl_correlation())


In [85]:
nfl_correlation()

0.0042821414363930855

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [115]:
from scipy.stats import ttest_rel
import pandas as pd
import numpy as np

def sports_team_performance():
    # Step 1: Group each sport by metro and average W-L%
    nfl_df = merged_NFL.groupby("Metropolitan area")["W-L%"].mean()
    nba_df = merged_nba.rename(columns={"W/L%": "W-L%"}).groupby("Metropolitan area")["W-L%"].mean()
    nhl_df = merged_Nhl.groupby("Metropolitan area")["W-L%"].mean()
    mlb_df = merged_Mlb.groupby("Metropolitan area")["W-L%"].mean()

    # Step 2: Organize in a dict
    sports_data = {
        "NFL": nfl_df,
        "NBA": nba_df,
        "NHL": nhl_df,
        "MLB": mlb_df
    }

    sports = list(sports_data.keys())
    p_values = pd.DataFrame(np.nan, index=sports, columns=sports)

    # Step 3: Compare each pair with ttest_rel
    for sport1 in sports:
        for sport2 in sports:
            if sport1 == sport2:
                continue

            # Get data only for shared cities
            shared_cities = sports_data[sport1].index.intersection(sports_data[sport2].index)
            perf1 = sports_data[sport1].loc[shared_cities]
            perf2 = sports_data[sport2].loc[shared_cities]

            # Perform paired t-test
            if len(perf1) > 1:  # At least 2 pairs required
                t_stat, p_val = ttest_rel(perf1, perf2)
                p_values.loc[sport1, sport2] = p_val
                p_values.loc[sport2, sport1] = p_val  # Symmetric

    # Assertions from assignment
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, f"NBA-NHL p-value ≠ 0.02 → Got {p_values.loc['NBA','NHL']:.4f}"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, f"MLB-NFL p-value ≠ 0.80 → Got {p_values.loc['MLB','NFL']:.4f}"

    return p_values


In [ ]:
sports_team_performance()